In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
from scipy import stats
from statsmodels.stats import weightstats as ws
from pydataset import data

In [2]:
df = data('iris')

# T-test

Assumptions: Independent samples are taken from a normal distribution. In this case, the normalized version of $\bar{X}$ using the sample standard deviation $S$ instead of the population standard deviation $\sigma$ follows the $t$-distribution with $n-1$ Degree of Freedom (dof).
- It is robust to the normality assumption

Test statistic: $T=\frac{\bar{X}-\mu_0}{S/\sqrt{n}}$



## 1-sample *t*-test

Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\mu=5$
- $H_a$: $\mu\neq5$

In [3]:
pg.ttest(
    x=df['Sepal.Length'], 
    y=5, 
    paired=False,
    tail='two-sided', #'greater', 'less'
    correction='auto', #True, False
    confidence=0.95
)

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,12.473257,149,two-sided,6.670742e-25,"[5.71, 5.98]",1.018437,5.88e+21,1.0


## 2-sample *t*-test

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $D_0 = \mu_1 − \mu_2 = 0$
- $H_a$: $D_a = \mu_1 − \mu_2 \neq 0$

In [4]:
pg.ttest(
    x=df.loc[df['Species']=='setosa','Sepal.Length'], 
    y=df.loc[df['Species']=='versicolor','Sepal.Length'],
    paired=False, # True for paired samples
    tail='two-sided', #'greater', 'less'
    correction='auto', #True, False
    confidence=0.95
)

,T,dof,tail,p-val,CI95%,cohen-d,BF10,power
T-test,-10.520986,98,two-sided,8.985235e-18,"[-1.11, -0.75]",2.104197,4.19e+14,1.0


# *Z*-Test

Assumptions: When sample size is large, the distribution of sample mean $(\bar{X})$ is approximately normal with
- $E(\bar{X}) = \mu$ 
- $Var(\bar{X}) = \sigma^2/n$

Test statistic: $Z=\frac{\bar{X}-\mu_0}{\sigma/\sqrt{n}}$

## 1-sample *Z*-test

Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\mu=5$
- $H_a$: $\mu\neq5$

In [5]:
zstat, pval = ws.ztest(
    x1=df['Sepal.Length'], 
    x2=None, 
    value=5.8,
    alternative='two-sided' #'larger', 'smaller'
)
print("z-stat:", zstat, "p-value:", pval)

z-stat: 0.6409183514112012 p-value: 0.5215757321117633


## 2-sample *Z*-test

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $D_0 = \mu_1 − \mu_2 = 0$
- $H_a$: $D_a = \mu_1 − \mu_2 \neq 0$

In [7]:
zstat, pval = ws.ztest(
    x1 = df.loc[df['Species']=='setosa','Sepal.Length'], 
    x2 = df.loc[df['Species']=='versicolor','Sepal.Length'],
    value = 0,
    alternative = 'two-sided' #'larger', 'smaller'
)
print("z-stat:", zstat, "p-value:", pval)

z-stat: -10.52098626754911 p-value: 6.914595261207391e-26


# Chi-Square test

## Hypotheses Concerning a Population Variance

- Assumption: Independent samples are taken from a normal distribution. In this case, $\chi^2 = \frac{(n-1)S^2}{\sigma^2}$
    - not robust to the normality assumption
- Test Statistic: $\chi^2 = \frac{(n-1)S^2}{\sigma_0^2}$
- Population: Sepal.Length of all 3 species (without differentiation among species).
- $H_0$: $\sigma=0.6857$
- $H_a$: $\sigma\neq0.6857$

In [9]:
def calculate_chi2_stat(x, var0):
    return (len(x)-1)*np.var(x)/var0

def var_test(x, var0, tail='two-sided'):
    chi2_stat = calculate_chi2_stat(x, var0)
    left_p = stats.chi2.cdf(chi2_stat, df=len(x)-1) # P(X<=x)
    if tail=='greater':
        return chi2_stat, 1-left_p
    if tail=='less':
        return chi2_stat, left_p
    else:
        return chi2_stat, (left_p if left_p<0.5 else 1-left_p) * 2

In [10]:
chi2_stat, pval = var_test(
    x = df['Sepal.Length'], 
    var0 = 1,
    tail='two-sided'
)
print("chi2-stat:", chi2_stat, "p-value:", pval)

chi2-stat: 101.48721111111111 p-value: 0.002101176207511642


## Comparison of Two Variances

- Assumption: Independent samples are taken from a normal distribution. In this case: $F = \frac{(n-1)S_1^2/\sigma_1^2}{(n-1)S_2^2/\sigma_2^2} = \frac{S_1^2/\sigma_1^2}{S_2^2/\sigma_2^2}$
    - not robust to the normality assumption
- Test Statistic: $F = \frac{S_1^2}{S_2^2}$

- Population 1: Sepal.Length of setosa
- Population 2: Sepal.Length of versicolor
- $H_0$: $\sigma_1=\sigma_2$
- $H_a$: $\sigma_1\neq\sigma_2$

In [11]:
def var_F_test(x, y, tail='two-sided'):
    f_stat = np.var(x)/np.var(y)
    left_p = stats.f.cdf(f_stat, dfn=len(x)-1, dfd=len(y)-1) # P(X<=x)
    if tail=='greater':
        return f_stat, 1-left_p
    if tail=='less':
        return f_stat, left_p
    else:
        return f_stat, (left_p if left_p<0.5 else 1-left_p) * 2

In [12]:
f_stat, pval = var_F_test(
    x = df.loc[df['Species']=='setosa','Sepal.Length'], 
    y = df.loc[df['Species']=='versicolor','Sepal.Length'],
    tail = 'two-tail'
)
print("F-stat:", f_stat, "p-value:", pval)

F-stat: 0.4663429131686986 p-value: 0.008657188362699816


## Goodness-of-Fit Test (Categorical Data)

- Rule of thumb: All expected cell counts are at least five.
- For an $r*c$ contingency table, the Test Statistic, $\chi^2 = \sum_{j=1}^{r} \sum_{i=1}^{c} \frac{[n_{ij}-E(n_{ij})]^2}{E(n_{ij})}$
    - where $E(n_{ij}) = \frac{row\_total*column\_total}{n}$
- dof: The number of cells minus 1 df for each independent linear restriction or each parameter estimated using MLE.
    - For an $r*c$ contingency table, $dof = (r-1)(c-1)$
- Variables:
    - Categorical variable: Species
    - Target variable: sepal_width_atleast3.1
- $H_0$: The categorical variables are not correlated.
- $H_a$: The categorical variables are correlated.

In [9]:
df['sepal_width_atleast3_1'] = df['Sepal.Width']>=3.1
df.groupby('Species')['sepal_width_atleast3.1'].value_counts()

Species     sepal_width_atleast3.1
setosa      True                      42
            False                      8
versicolor  False                     42
            True                       8
virginica   False                     33
            True                      17
Name: sepal_width_atleast3.1, dtype: int64

In [123]:
expected, observed, stats = pg.chi2_independence(df, x='Species', y='sepal_width_atleast3.1')
display(expected.style.set_caption("Expected"), observed.style.set_caption("observed"), stats)

sepal_width_atleast3.1,False,True
Species,,
setosa,27.666667,22.333333
versicolor,27.666667,22.333333
virginica,27.666667,22.333333


sepal_width_atleast3.1,False,True
Species,,
setosa,8,42
versicolor,42,8
virginica,33,17


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,50.224780,2.0,1.241160e-11,0.578647,0.999999
1,cressie-read,0.666667,50.918092,2.0,8.775601e-12,0.582627,0.999999
2,log-likelihood,0.000000,54.196713,2.0,1.703466e-12,0.601092,1.000000
3,freeman-tukey,-0.500000,58.722124,2.0,1.772771e-13,0.625684,1.000000
4,mod-log-likelihood,-1.000000,65.637935,2.0,5.583462e-15,0.661503,1.000000
5,neyman,-2.000000,90.663462,2.0,2.054373e-20,0.777447,1.000000


# ANOVA

## One-way ANOVA
- Assumption: The $k$ populations are normally distributed with a same variance.
    - The test is still robust with moderate departures from the normality and equal variances assumption.
    - The equal variances assumption is less critical if sample size of all populations are equal.
- Test Statistic: $F = MST/MSE$, large F indicates that $H_0$ should be rejected.
- Variables:
    - Categorical variable 1: Species
    - Categorical variable 2: sepal_width_atleast3.1
    - Target variable: Sepal.Length
- $H_0$: $\mu_1 = \mu_2 = ... = \mu_n$
- $H_a$: At least 1 of the population mean is significantly different from another.

In [5]:
pg.anova(data=df, dv='Sepal.Length', between='Species', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,Species,63.212133,2,31.606067,119.264502,1.669669e-31,0.618706
1,Within,38.956200,147,0.265008,NaN,NaN,NaN


## Two-way ANOVA

Similar to one-way ANOVA but it handles two categorical variables and their interactions, each variable and the interaction have a *p*-value to differentiate if the variable/interaction lead to different means.

In [14]:
pg.anova(
    data=df.rename(columns={'Sepal.Length':'sepal_len'}), 
    dv='sepal_len', 
    between=['Species','sepal_width_atleast3_1'], 
    detailed=True
)

,Source,SS,DF,MS,F,p-unc,np2
0,Species,65.086000,2.0,32.543000,139.705044,1.884920e-34,0.659904
1,sepal_width_atleast3_1,4.963994,1.0,4.963994,21.310114,8.574314e-06,0.128910
2,Species * sepal_width_atleast3_1,0.448736,2.0,0.224368,0.963197,3.841159e-01,0.013201
3,Residual,33.543470,144.0,0.232941,NaN,NaN,NaN
